<a href="https://colab.research.google.com/github/miladmirzazadeh/Gamein-Prototype/blob/master/prototype_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
class Material():
  Prices = _ #dict (id , price)

In [ ]:
dic = dict({"1":"hi"})
dic[list(dic.keys())[0]]

'hi'

In [ ]:
x = pd.Series()
x["ali"] = 110
x["milad"] = 200
x= x.sort_values(ascending=False)
x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


milad    200
ali      110
dtype: int64

In [ ]:
class Controller():
  return_message=""
  turn = 0 
  last_turn = 50
  player_objects = dict()#name,object

  def next_turn():
    turn+=1
    if turn == Controller.last_turn : 
      Controller.return_message = "End Game. Thank u "
      score_board = Controller.score_board()

  
  def score_board():
    net_worths = pd.Series([])
    for player in list(player_objects.keys()):
      obj = player_objects[player]
      obj.calculate_final_net_worth()
      net_worths[player] = obj 
    net_worths = net_worths.sort(ascending=False)
    return net_worths





  def get_status(player_name):
    pass
  def calculate_net_worth(player_name):
    pass
  def create_player(player_name):
    land_id = _#random از بین لیست زمین های در دسترس
    #remove from available lands
    Controller.player_objects.update({name : Player(name, land_id)})
    
  def buy_land(player_name, land_id):
    Controller.player_objects[name].add_land(land_id)
    # Controller.player_objects[name].cash -=         اینجا باید پول زمین رو از حسابش کم کنیم . فعلا لازم نیست  چون خودمون بهش زمین رو اختصاص میدیم 
    
    


  def upgrade_stockhouse(player_name, land_id):
    pass
  def buy_machine(player_name, land_id, machine_id):
    pass
  def show_product_market_info():
    pass
  def buy_product(player_name, market_row): # شماره ردیف محصول مد نظر در ویترین مارکت به عنوان ورودی داده میشود
    pass

In [ ]:
Controller.create_player("Milad")
Controller.create_player("Pouria")
Controller.create_player("Faezeh")

NameError: ignored

In [ ]:
class Player():
  players_name=[]

  def __init__(self, name, land_id):
    self.name = name
    #remove from available
    self.cash = 1000000 # بودجه اولیه
    self.land_objects=dict()

  def calculate_final_net_worth(self):
    inventory_value_list = []
    for land in list(self.land_objects.keys()):
      inventory_value_list.append(self.land_objects[land].calculate_inventory_value()) # تو هر خونه این لیست ، ارزش دارایی های یک زمین از زمین های بازیکن ثبت میشه
    inventory_value_sum = sum(inventory_value_list)
    net_worth = inventory_value_sum + self.cash 
    return net_worth 

  def add_land(self, land_id):
    self.land_objects.update({land_id, Land(land_id)}) # یه اینستنس از لند میسازه که ورودی بهش آیدی میدیم . بعد توی خود لند ، بر حسب آیدی بقیه فیچر های لند رو ست میکنیم
    

      



    print("hi")



In [ ]:
class Land():
  lands_list =[] #amirreza یه دیتا فریم که یه آی دی زمین داره و ماشین
  available_lands=[] #آی دی های باقی مانده
  def __init__(self):
    self.stockhouse = StockHouse()
    


  

In [ ]:
class StockHouse():
  def __init__(self):
    self.remaining_cap = 3000
    self.inventory_df = pd.DataFrame()
    self.cost_per_product = 100

  

In [ ]:
class Machine() 

In [ ]:
class Product():
  

In [ ]:
ali = Player("aliagha",1)
milad = Player("mili",1)

In [ ]:
Player.players_name.append(ali.name)
Player.players_name.append(milad.name)

In [ ]:
ali.calculate_net_worth()

UnboundLocalError: ignored

In [ ]:
class a():
  def x_2(): 
    print("hi")
  def y():
    a.x_2()

In [ ]:
a.y()

hi
